In [1]:
import re
import time
import pandas as pd

from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import  NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from browser import getBrowserDriver

browser, waiter = getBrowserDriver()

# Pagina Inicial
el_extrajudicial = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/div/div/div/ul/li[2]/a')))
el_extrajudicial.click()

el_serventias_extrajudicial = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/div/div/div/ul/li[2]/ul/li[1]/a')))
el_serventias_extrajudicial.click()

# Criando dataframe
data = {
    'Denominacao': [""],
    'Situacao': [""],
    'Atribuicoes': [""],
    'Responsavel': [""],
    'Substituto': [""],
    'UF': [""],
    'Municipio': [""],
    'Telefone Principal': [""],
    'Telefone Secundario': [""],
    'Email': [""],
}

df = pd.DataFrame(data)

In [2]:
def getEstados():
    el_mapa_ufs_geral = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset/map')))
    el_mapa_ufs = el_mapa_ufs_geral.find_elements(By.TAG_NAME, "area")

    return el_mapa_ufs

def getMunicipios():
    el_mapa_muns_geral = waiter.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="cidade_serventia"]')))
    el_mapa_muns = el_mapa_muns_geral.find_elements(By.TAG_NAME, "option")

    return el_mapa_muns

def getDados():
    #Dados cartorio
    try:
        denominacao = browser.find_element(By.XPATH, '/html/body/div[2]/div[5]/fieldset[1]/table/tbody/tr[2]/td[2]').text
    except NoSuchElementException:
        denominacao = None
    
    try:
        situacao = browser.find_element(By.XPATH, '/html/body/div[2]/div[5]/fieldset[1]/table/tbody/tr[5]/td[2]').text
    except NoSuchElementException:
        situacao = None

    #Atribuições
    try:
        atribuicoes = ""
        element_temp = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset[2]/table/tbody')))
        elements_temp = element_temp.find_elements(By.TAG_NAME, 'td')
        for c in range(0, len(elements_temp)):
                if(c % 2 == 1 and c != 1):
                    atribuicoes += f"{elements_temp[c].text[1:]} ;"
    except:
        atribuicoes = None

    #Responsaveis
    try:
        responsavel = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset[3]/table/tbody/tr[1]/td[2]'))).text
        responsavel = re.sub(r"\n.*", "", responsavel)
    except:
        responsavel = None

    try:
        substituto = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset[3]/table/tbody/tr[3]/td[2]'))).text
        substituto = re.sub(r"\n.*", "", substituto)
    except:
        substituto = None

    #Localização
    try:
        uf = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset[4]/table/tbody/tr[1]/td[2]'))).text
    except:
        uf = None
    
    try:
        municipio = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset[4]/table/tbody/tr[2]/td[2]'))).text
    except:
        municipio = None
    
    try:
        telefone_principal = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset[4]/table/tbody/tr[6]/td[2]'))).text
    except:
        telefone_principal = None
    
    try:
        telefone_secundario = browser.find_element(By.XPATH, '/html/body/div[2]/div[5]/fieldset[4]/table/tbody/tr[7]/td[2]').text
    except NoSuchElementException:
        telefone_secundario = None

    try:
        email = browser.find_element(By.XPATH, '/html/body/div[2]/div[5]/fieldset[4]/table/tbody/tr[7]/td[2]').text
    except NoSuchElementException:
        email = None

    #Atribuições
    try:
        atribuicoes = ""
        element_temp = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset[2]/table/tbody')))
        elements_temp = element_temp.find_elements(By.TAG_NAME, 'td')
        for c in range(0, len(elements_temp)):
                if(c % 2 == 1 and c != 1):
                    atribuicoes += f"{elements_temp[c].text[1:]} ;"
    except:
        atribuicoes = None

    #Arrecadações
    try:
        arrecadacoes = []
        datas = []
        element_temp = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset[6]/table[2]/tbody')))
        elements_temp = element_temp.find_elements(By.TAG_NAME, 'td')
        i = 0
        for c in range(0, len(elements_temp)):
            i += 1
            if i == 3:
                arrecadacoes.append(elements_temp[c].text)
                i = 0
            elif i == 1:
                titulo = elements_temp[c].text
                datasss = re.findall(r'\d{2}/\d{2}/\d{4}', titulo)
                ultima_data = datasss[-1] if datasss else None
                datas.append(ultima_data)

    except:
        arrecadacoes = []

    element_temp = browser.find_element(By.XPATH, '/html/body/div[2]/div[5]/fieldset[4]/table/tbody')
    element_temp = element_temp.find_elements(By.TAG_NAME, 'tr')

    if len(element_temp) == 7:
        email = telefone_secundario
        telefone_secundario = None

    nova_linha = {
        'Denominacao': denominacao,
        'Situacao': situacao,
        'Atribuicoes': atribuicoes,
        'Responsavel': responsavel,
        'Substituto': substituto,
        'UF': uf,
        'Municipio': municipio,
        'Telefone Principal': telefone_principal,
        'Telefone Secundario': telefone_secundario,
        'Email': email,
        **{f'{datas[i]}': valor for i, valor in enumerate(arrecadacoes)}  # Colunas de arrecadação
    }

    nova_linha_df = pd.DataFrame([nova_linha])  # Criar um novo DataFrame com a nova linha

    return nova_linha_df

In [3]:
# Pagina Mapa
error = False
el_mapa_ufs = getEstados()

# Selecionando estados
for uf in range(1, 27):
    # for num in range(1, 2):
    num = 1
    while(True):
        try:
            el_mapa_ufs = getEstados()
            el_mapa_ufs[uf].click()
            el_mapa_muns = getMunicipios()
            el_mapa_muns[num].click()

            browser.implicitly_wait(3)
            el_pesquisar = waiter.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="div_cidade"]/div/table/tbody/tr[2]/td/button[1]')))
            el_pesquisar.click()

            # Pagina Cartorios
            # Selecionando x quantidade de dados em um municipio
            try:
                el_infos_geral = waiter.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="display"]/tbody')))
                el_infos = el_infos_geral.find_elements(By.TAG_NAME, 'a')

                for info in range(len(el_infos) - 1, len(el_infos)):
                    el_infos[info].click()
                    try:
                        el_infos[info].click()
                    except StaleElementReferenceException:
                        pass

                    browser.implicitly_wait(1)
                    nova_linha_df = getDados()
                    df = pd.concat([df, nova_linha_df], ignore_index=True)  # Concatenate DataFrames

                    browser.back()
                    browser.implicitly_wait(1)

                    el_infos_geral = waiter.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="display"]/tbody')))
                    el_infos = el_infos_geral.find_elements(By.TAG_NAME, 'a')
            except:
                error = True

            # Resetando para conseguir clicar novamente
            browser.back()
            browser.back()

            if(error):
                el_serventias_extrajudicial = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/div/div/div/ul/li[2]/ul/li[1]/a')))
                el_serventias_extrajudicial.click()

            browser.implicitly_wait(1)
            error = False
        except StaleElementReferenceException:
            ...
        
        num += 1

        if(num == len(el_mapa_muns)):
            break

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=115.0.5790.173)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00007FF695334A62+57106]
	(No symbol) [0x00007FF6952ACF52]
	(No symbol) [0x00007FF69517E2CB]
	(No symbol) [0x00007FF69515DEB9]
	(No symbol) [0x00007FF6951DAE07]
	(No symbol) [0x00007FF6951EDBEF]
	(No symbol) [0x00007FF6951D6793]
	(No symbol) [0x00007FF6951ACE81]
	(No symbol) [0x00007FF6951AE064]
	GetHandleVerifier [0x00007FF6955E4222+2873042]
	GetHandleVerifier [0x00007FF695636590+3209792]
	GetHandleVerifier [0x00007FF69562F3AF+3180639]
	GetHandleVerifier [0x00007FF6953C5F25+652245]
	(No symbol) [0x00007FF6952B8618]
	(No symbol) [0x00007FF6952B47C4]
	(No symbol) [0x00007FF6952B48BC]
	(No symbol) [0x00007FF6952A4C33]
	BaseThreadInitThunk [0x00007FFF89E726AD+29]
	RtlUserThreadStart [0x00007FFF8B8EAA68+40]


In [ ]:
# # Pagina Mapa

# # Selecionando estados
# el_mapa_ufs_geral = waiter.until(EC.presence_of_element_located((By.XPATH, r'/html/body/div[2]/div[5]/fieldset/map')))
# el_mapa_ufs = el_mapa_ufs_geral.find_elements(By.TAG_NAME, "area")

# el_mapa_ufs[-10].click()

# # Selecionando municipios
# el_mapa_muns_geral = waiter.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="cidade_serventia"]')))
# el_mapa_muns = el_mapa_muns_geral.find_elements(By.TAG_NAME, "option")

# el_mapa_muns[19].click()

# # Pesquisar
# el_pesquisar = waiter.until(EC.presence_of_element_located((By.XPATH, r'//*[@id="div_cidade"]/div/table/tbody/tr[2]/td/button[1]')))
# el_pesquisar.click()

In [4]:
df = df.drop(index = 0)

In [5]:
df

,Denominacao,Situacao,Atribuicoes,Responsavel,Substituto,UF,Municipio,Telefone Principal,Telefone Secundario,Email,...,31/12/2018,30/06/2019,31/12/2019,30/06/2020,31/12/2020,30/06/2021,31/12/2021,30/06/2022,31/12/2022,30/06/2023
1,ESCRIVANIA DE PAZ DE ABDON BATISTA,PROVIDO,Registro Civil das Pessoas Naturais e de Inter...,MARIA AMÁLIA ALDANA FARIA,DIENEFER BORGES DE ARAUJO,SC,ABDON BATISTA,(49)3545-1011,(49)8888-4114,epazabdonbatista@yahoo.com.br,...,"R$ 75.207,21","R$ 82.640,66","R$ 116.575,97","R$ 103.448,26","R$ 120.829,17","R$ 89.615,26","R$ 126.110,13","R$ 126.224,52","R$ 154.244,07","R$ 115.977,11"
2,Serviço Distrital de Abatiá,PROVIDO,Registro Civil das Pessoas Naturais e de Inter...,JOSé AUGUSTO ARRUDA NETO,Não possui Substituto informado.,PR,ABATIÁ,(43)3556-1263,None,joseaugusto@cartorioabatia.com.br,...,"R$ 93.424,31","R$ 87.219,28","R$ 104.642,85","R$ 84.844,60","R$ 142.754,31","R$ 157.792,68","R$ 177.974,65","R$ 186.178,64","R$ 182.569,43",Pendente
3,"2º CARTÓRIO DE NOTAS, DA COMARCA DE ADAMANTINA",PROVIDO,None,None,None,SP,ADAMANTINA,,None,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RCPN DO 1º DISTRITO DE ANGRA DOS REIS,PROVIDO,,RAQUEL VIEIRA ABRÃO REZENDE,Não possui Substituto informado.,RJ,ANGRA DOS REIS,(24)3365-4979,None,administrativo@rcpn1angra.com.br,...,"R$ 288.284,31","R$ 262.297,10","R$ 542.661,22","R$ 445.310,58","R$ 516.186,56","R$ 621.999,39","R$ 717.186,12","R$ 728.318,28","R$ 680.485,02","R$ 754.356,79"
5,Cartório de Registro Civil e Tabelionato - Dis...,VAGO,None,None,None,ES,AFONSO CLAUDIO,,None,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Ofício do Registro Civil e Tabelionato de Nota...,PROVIDO,Registro Civil das Pessoas Naturais e de Inter...,MARCIA CORTES ROCHA,DJEINY OLIVEIRA LUIZ,MG,ABADIA DOS DOURADOS,(34)3847-1235,(34)98829-2854,cartorio.abadia@uol.com.br,...,"R$ 232.016,36","R$ 336.189,31","R$ 262.800,58","R$ 219.150,51","R$ 321.374,91","R$ 462.941,10","R$ 504.924,25","R$ 424.274,43","R$ 603.425,96","R$ 529.899,07"
7,CARTÓRIO DO REGISTRO CIVIL COM FUNÇÕES NOTARIA...,INATIVA,Registro Civil das Pessoas Naturais e de Inter...,MARCIO BARROS DE OLIVEIRA,Não possui Substituto informado.,BA,ABAÍRA,(77)3479-2102,None,omesquita@tjba.jus.br,...,Pendente,Pendente,Pendente,Pendente,Pendente,Pendente,Pendente,Pendente,Pendente,Pendente


In [ ]:
df.to_excel("scraping.xlsx")
